In [9]:
import numpy as np
import cv2 as cv
import matplotlib as plt
img = cv.imread('images/baggage_claim.jpg')
gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)


In [10]:
sift = cv.xfeatures2d.SIFT_create()


In [12]:
kp = sift.detect(gray,None)
img=cv.drawKeypoints(gray,kp,img)
cv.imshow(np.asarray(img))
cv.imshow

TypeError: imshow() missing required argument 'mat' (pos 2)

In [6]:
kp

[<KeyPoint 00000242A4293CF0>,
 <KeyPoint 00000242B39E4240>,
 <KeyPoint 00000242B39E4270>,
 <KeyPoint 00000242B39E42A0>,
 <KeyPoint 00000242B39E42D0>,
 <KeyPoint 00000242B39E4300>,
 <KeyPoint 00000242B39E4330>,
 <KeyPoint 00000242B39E4360>,
 <KeyPoint 00000242B39E43C0>,
 <KeyPoint 00000242B39E43F0>,
 <KeyPoint 00000242B39E4420>,
 <KeyPoint 00000242B39E4060>,
 <KeyPoint 00000242B39E4480>,
 <KeyPoint 00000242B39E45A0>,
 <KeyPoint 00000242B39E4630>,
 <KeyPoint 00000242B39E4660>,
 <KeyPoint 00000242B39E46C0>,
 <KeyPoint 00000242B39E46F0>,
 <KeyPoint 00000242B39E4720>,
 <KeyPoint 00000242B39E4750>,
 <KeyPoint 00000242B39E4780>,
 <KeyPoint 00000242B39E47B0>,
 <KeyPoint 00000242B39E44B0>,
 <KeyPoint 00000242B39E4810>,
 <KeyPoint 00000242B39E4390>,
 <KeyPoint 00000242B39E4570>,
 <KeyPoint 00000242B39E4870>,
 <KeyPoint 00000242B39E45D0>,
 <KeyPoint 00000242B39E4600>,
 <KeyPoint 00000242B39E48A0>,
 <KeyPoint 00000242B39E48D0>,
 <KeyPoint 00000242B39E4690>,
 <KeyPoint 00000242B39E4900>,
 <KeyPoint